# Visualización de datos en Altair

[Altair](https://altair-viz.github.io/index.html) is a declarative statistical visualization library for Python, based on Vega and Vega-Lite, and the source is available on GitHub.

With Altair, you can spend more time understanding your data and its meaning. Altair’s API is simple, friendly and consistent and built on top of the powerful Vega-Lite visualization grammar. This elegant simplicity produces beautiful and effective visualizations with a minimal amount of code. Altair offers a powerful and concise visualization grammar that enables you to build a wide range of statistical visualizations quickly.

Qué significa [**declarativo (declarative)**](https://en.wikipedia.org/wiki/Declarative_programming)
Declarativo  quiere decir que esta lenguaje se enfoca en **qué** queremos y no en cómo lo tendremos (no tenemos que escribir instrucciones detalladas de cómo hará el programa para graficar nuestros datos). 

Está basado en la gramática de los gráficos: 

A grammar of graphics is a tool that enables us to concisely describe the components of a graphic. Such a grammar allows us to move beyond named graphics (e.g., the “scatterplot”) and gain insight into the deep structure that underlies statistical graphics. [(Wickham, 2010)](https://byrneslab.net/classes/biol607/readings/wickham_layered-grammar.pdf).

Los elementos que componen al GoG son:
- Datos
- Coordenadas
- Escalas
- Estética
- Geometrías
- Estadísticas
- Facetas 


https://towardsdatascience.com/a-comprehensive-guide-to-the-grammar-of-graphics-for-effective-visualization-of-multi-dimensional-1f92b4ed4149




In [1]:
import pandas as pd
import altair as alt

In [2]:
#pip install altair

Vamos a trabajar con los datos del último [reporte de felicidad](https://www.kaggle.com/ajaypalsinghlo/world-happiness-report-2021/code). Altair asume que nuestra base de datos es _ordenada_("tidy")<sup>1</sup>.


<sup>1</sup>Las bases de datos _tidy_ son fáciles de manejar, modelar y visualizar, y tienen una estructura específica: cada variable es una columna, cada observación es una fila, y cada tipo de unidad observacional es una tabla. [(Wickham, 2014)](https://www.jstatsoft.org/article/view/v059i10)

In [3]:
happy_all = pd.read_csv("./data/world-happiness-report-2023.csv")
happy_23 = pd.read_csv("./data/world-happiness-report-2023-countries.csv")

happy_21 = pd.read_csv("./data/world-happiness-report-2021.csv")

In [4]:
#happy_23.head(20)

h_all_22 = happy_all[happy_all['year'] == 2022]



#happy_21[['Country name', "Regional indicator" ]]

In [5]:
happy = pd.merge(h_all_22,
                  happy_21[['Country name', "Regional indicator" ]],
                  on = "Country name",
                  #how  = "left",
                 how  = "inner",
                 indicator = True)

In [6]:
print(h_all_22.shape)
print(happy_21.shape)
print(happy.shape)




(114, 11)
(149, 20)
(111, 13)


## El objeto `Chart`

El objeto más importante de Altair es el `Chart`, que toma como único argumento un dataframe. 


In [7]:
#alt.Chart(happy)



Ya podemos empezar a decidir nuestro gráfico. Primero queremos decidir qué marcas (marks) geométricas queremos que tenga. Elegimos el atributo `mark_point()`. 
Así como `point`, existen otros tipos de _marks_
- point
- line
- bar
- area
- rect
- geoshape
- text
- circle
- square
- rule
- tick

In [8]:
alt.Chart(happy).mark_point()

alt.Chart(...)

Aquí todavía no estamos dando una ubicación a los datos, así que está graficando todo en un mismo punto. Sin embargo, podemos especificar posiciones para nuestos puntos. Para ello usaremos el método `encode` para darle _encodings_ visuales a nuestos datos. En este caso, en el eje y colocaremos a los países de Latinoamérica.

In [9]:
happy_lac = happy[happy['Regional indicator'] == "Latin America and Caribbean"]

In [10]:
alt.Chart(happy_lac).mark_point().encode(
x = "Country name", 

)

alt.Chart(...)

El método del encoding crea un un mapeo de key-value entre varios encodings (x, y, color, forma, tamaño, etc) a las variables de nuestra base de datos, tan sólo usando sus nombres. Noten que Altair detecta el tipo de datos de un dataframe de pandas.  Aún tenemos varios puntos que se superponen, los separaremos agregando un encoding para x. 


In [11]:
alt.Chart(happy_lac).mark_point().encode(
y = "Country name", 
    x = "Life Ladder"
    )

alt.Chart(...)

In [12]:
#happy_lac[['Life Ladder', 'Country name']]

Una forma más generalizada de realizar este mismo códig, es agregar `alt.X`  y `alt.Y`, lo cual permitirá agregar más parámetros, y por tanto, más personalización a nuestro gráfico. 

In [13]:
alt.Chart(happy_lac).mark_point().encode(
x = alt.X("Country name" ), 
y = alt.Y("Life Ladder")
)

alt.Chart(...)

¿ Qué pasa si queremos un gráfico de barras?

In [14]:
alt.Chart(happy_lac).mark_bar().encode(
y = alt.Y("Life Ladder"),
x = alt.X("Country name")
)

alt.Chart(...)

In [15]:
alt.Chart(happy_lac).mark_point().encode(
    alt.X("Life Ladder"),
    alt.Y("Country name")
)

alt.Chart(...)

A veces es necesario decirle explícitamente a Altair el tipo de datos de nuestras variables. 
Estos tipos pueden ser/se especifican de la siguiente forma:

- `'nombre_variable:N'` Tipo de datos Nominal (datos categóricos no ordenados),

- `'nombre_variable:O'` Tipo de datos Ordinal (datos categóricos ordenados),

- `'nombre_variable:Q'` Tipo de datos cuantitativo (datos numéricos) y

- `'nombre_variable:T'` Variables de tiempo como fechas o años (casi no hemos visto esto)

Volviendo al primer ejemplo:

In [16]:
alt.Chart(happy_lac).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Country name:N")
    )

alt.Chart(...)

En el siguiente ejemplo haremos un scatterplot (gráfico de dispersión) de la relación entre felicidad y PBI:

In [17]:
alt.Chart(happy).mark_point().encode(
    y = alt.Y("Log GDP per capita:Q"), # pbi per capita
    x = alt.X("Life Ladder:Q"), # felicidad
    

    )

alt.Chart(...)

Podemos agregar un encoding adicional: El color. En este caso, color significará la región de pertenencia. 

In [18]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = "Regional indicator:N"
    )
# Notar que  las regiones son variables nominales, por ello elige diferentes colores como encoding. 

alt.Chart(...)

¿Qué pasa si coloreamos por una variable cuantitativa?

In [19]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = 'Healthy life expectancy at birth'
    )

alt.Chart(...)

Agregando interactividad básica:

In [20]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = "Regional indicator"
    ).interactive()


alt.Chart(...)

Qué pasaría si queremos agregar el _tooltip_ 

In [21]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = "Regional indicator",
    tooltip = ["Country name", "Life Ladder"]
    ).interactive()

alt.Chart(...)

Qué pasa si queremos agregar un gráfico por región:

In [22]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = alt.Color("Regional indicator"),
    column = alt.Column("Regional indicator")
    )

alt.Chart(...)

In [23]:
alt.Chart(happy).mark_point(filled = True).encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = alt.Color("Regional indicator", 
            legend=alt.Legend(orient='bottom', titleOrient='left')),
    column = alt.Column("Regional indicator")
    ).properties(width=150, height=150)

alt.Chart(...)

Qué pasa si queremos agregar un encoding más respecto al tamaño de los puntos:

In [24]:
alt.Chart(happy).mark_point().encode(
    x = alt.X("Life Ladder:Q"),
    y = alt.Y("Log GDP per capita:Q"),
    color = alt.Color("Regional indicator"),
    size = alt.Size("Generosity"),
    tooltip = ["Country name", "Life Ladder"]

    ).interactive()

alt.Chart(...)

In [25]:
happy_peru = happy_all[happy_all['Country name'] == 'Peru']

In [26]:
elem1 = alt.Chart(happy_peru).mark_line().encode(
x = alt.X('year'),
y = alt.Y('Life Ladder:Q')
)

elem1

alt.Chart(...)

In [27]:
elem2 = alt.Chart(happy_peru).mark_area(color='blue', opacity = 0.5).encode(
x = alt.X('year'),
y = alt.Y('Life Ladder:Q')
)

#

elem2

alt.Chart(...)

In [28]:
elem1 + elem2

alt.LayerChart(...)